In [ ]:
# Limpieza de comentarios
import pandas as pd
import re

def clean_comment(comment):
    if pd.isnull(comment):  # Si es NaN, devolvemos string vacío
        return ''
    # Eliminar URLs
    comment = re.sub(r'http[s]?://\S+', '', comment)
    # Emoticones comunes
    comment = re.sub(r'[:;=][-~]?[)D]', '', comment) 
    # Eliminar caracteres no alfabéticos o números
    comment = re.sub(r'[^a-zA-Z\s]', '', comment)
    # Eliminar palabras demasiado cortas (como "a", "b", "c")
    comment = re.sub(r'\b\w{1,2}\b', '', comment)
    # Eliminar espacios múltiples
    comment = re.sub(r'\s+', ' ', comment).strip()
    return comment

def clean_comments(csv_path):
    # Cargar los datos de comentarios
    df = pd.read_csv(csv_path)
    
    # Limpiar los comentarios
    df['cleaned_comment'] = df['text'].apply(clean_comment)
    
    # Eliminar las filas donde el comentario está vacío después de la limpieza
    # También se eliminan los replies a comentarios vacíos.
    print("Cantidad de mensajes iniciales: ", df.shape[0])
    mask_empty = (df['cleaned_comment'].str.strip() == '')
    print("Comentarios vacíos: ", mask_empty.sum())
    video_ids_to_filter = df[mask_empty]["comment_id"]
    mask_replies = ((df['is_reply'] == True) & (df['reply_to_comment_id'].isin(video_ids_to_filter)))
    print("Replies a comentarios vacíos: ", mask_replies.sum())
    mask_to_remove = mask_empty | mask_replies
    df = df[~mask_to_remove] #Eliminación de mensajes vacíos o replies a mensajes vacíos
    print("Cantidad de mensajes finales: ", df.shape[0])
    
    # Guardar los resultados limpios en un nuevo archivo CSV
    #df.to_csv('../data/cleaned_comments.csv', index=False)
    print("Los comentarios han sido limpiados y guardados en '../data/cleaned_comments.csv'.")
    return df

cleaned_comments = clean_comments('../data/comments.csv')